Geolocation

In [ ]:
#Identify the geolocation of municipalities

import pandas as pd
import googlemaps


def obtener_latitud_longitud(lugar, api_key):
    gmaps = googlemaps.Client(key=api_key)
    try:
        
        resultado = gmaps.geocode(lugar)
        
        
        latitud = resultado[0]['geometry']['location']['lat']
        longitud = resultado[0]['geometry']['location']['lng']
        
        return latitud, longitud
    except Exception as e:
        print(f"Error al obtener la información para {lugar}: {e}")
        return None, None


archivo_excel_entrada = 'Ayuntamientos.xlsx'  
datos = pd.read_excel(archivo_excel_entrada)


clave_api_google = 'XXXXX-XXXXXX-XXXXXx'  


nombres_ayuntamiento = []
latitudes = []
longitudes = []


for index, fila in datos.iterrows():
    lugar = fila['Nombre del ayuntamiento']  
    latitud, longitud = obtener_latitud_longitud(lugar, clave_api_google)
    
    
    nombres_ayuntamiento.append(lugar)
    latitudes.append(latitud)
    longitudes.append(longitud)

    
    print(f"{lugar}: Latitud = {latitud}, Longitud = {longitud}")


coordenadas = pd.DataFrame({'Nombre del ayuntamiento': nombres_ayuntamiento, 'Latitud': latitudes, 'Longitud': longitudes})

archivo_excel_salida = 'coordenadas.xlsx'
coordenadas.to_excel(archivo_excel_salida, index=False)
print(f"Coordenadas guardadas en {archivo_excel_salida}")


Coordinates excel to GeoJason

In [15]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


df = pd.read_excel('coordenadas.xlsx')
df['geometry'] = df.apply(lambda row: Point(row['longitud'], row['latitud']), axis=1)
gdf = gpd.GeoDataFrame(df, geometry='geometry')


gdf.to_file('buscador.geojson', driver='GeoJSON')


Import graphics from City Hall

In [9]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd


ruta_gastos = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/Gastosos 2.xlsx'
gastos_df = pd.read_excel(ruta_gastos)
gastos_df = gastos_df.rename(columns={'GASTOS': 'MONTO'})



sns.set(style="whitegrid", palette="muted")


def money_formatter(x, pos):
    if x >= 1e6:  
        return '${:,.1f}M'.format(x * 1e-6)
    elif x >= 1e3:  
        return '${:,.0f}K'.format(x * 1e-3)
    else:
        return '${:,.0f}'.format(x)


ayuntamientos = gastos_df['AYUNTAMIENTO'].unique()

if not os.path.exists('graphs_ayuntamientos'):
    os.makedirs('graphs_ayuntamientos')


meses_orden = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", 
               "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]


gastos_df['MES'] = pd.Categorical(gastos_df['MES'], categories=meses_orden, ordered=True)

for i, ayuntamiento in enumerate(ayuntamientos, start=1):
    df_ayuntamiento = gastos_df[gastos_df['AYUNTAMIENTO'] == ayuntamiento]
    monthly_expenses = df_ayuntamiento.groupby('MES')['MONTO'].sum().reset_index()

    plt.figure(figsize=(12, 6))
    bar_plot = sns.barplot(x='MES', y='MONTO', data=monthly_expenses)
    plt.title(f'Gastos por Mes en {ayuntamiento}', fontsize=16)
    plt.xlabel('Mes', fontsize=14)
    plt.ylabel('Gastos Totales', fontsize=14)
    plt.xticks(rotation=45)

    
    plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(money_formatter))

   
    for p in bar_plot.patches:
        height = p.get_height()
        bar_plot.annotate(money_formatter(height, None), 
                          (p.get_x() + p.get_width() / 2., height), 
                          ha = 'center', va = 'center', 
                          xytext = (0, 9), 
                          textcoords = 'offset points')

    plt.tight_layout()
    filename = f'{str(i).zfill(2)}.jpg'  
    plt.savefig(f'graphs_ayuntamientos/{filename}')
    plt.close()

   

C:\Users\franc\AppData\Local\Temp\ipykernel_8996\3445314996.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_expenses = df_ayuntamiento.groupby('MES')['MONTO'].sum().reset_index()
C:\Users\franc\AppData\Local\Temp\ipykernel_8996\3445314996.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_expenses = df_ayuntamiento.groupby('MES')['MONTO'].sum().reset_index()
C:\Users\franc\AppData\Local\Temp\ipykernel_8996\3445314996.py:41: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior 

Expanded BoxPlot

In [4]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import numpy as np

ruta_gastos1 = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/Gastosos 2.xlsx'
gastos_df = pd.read_excel(ruta_gastos1)
gastos_df = gastos_df.rename(columns={'GASTOS': 'MONTO'})

sns.set(style="whitegrid", palette="muted")

class CustomFormatter(mticker.Formatter):
    def __call__(self, x, pos=None):
        if x >= 1e6:
            s = '{:1.1f}M'.format(x*1e-6)
        elif x >= 1e3:
            s = '{:1.0f}K'.format(x*1e-3)
        else:
            s = '{:1.0f}'.format(x)
        return s
    

def money_formatter(x, pos):
    if x >= 1e6:  
        return '${:,.1f}M'.format(x * 1e-6)
    elif x >= 1e3:  
        return '${:,.0f}K'.format(x * 1e-3)
    else:
        return '${:,.0f}'.format(x)

if not os.path.exists('graphs_ayuntamientos_boxplotParte3'):
    os.makedirs('graphs_ayuntamientos_boxplotParte3')

ayuntamientos = gastos_df['AYUNTAMIENTO'].unique()
ayuntamientos_a_omitir = [
    'Junta de Distrito Municipal de Arroyo Dulce', 
    'Junta de Distrito Municipal de Platanal'
]

contador_imagenes = 0
for ayuntamiento in ayuntamientos:
    contador_imagenes += 1 
    
    plt.figure(figsize=(12, 6))
    if ayuntamiento in ayuntamientos_a_omitir:
        plt.title(f'Box Plot de Gastos para {ayuntamiento}', fontsize=16)
        plt.axis('off')  
    else:
        df_ayuntamiento = gastos_df[gastos_df['AYUNTAMIENTO'] == ayuntamiento]
        if not df_ayuntamiento.empty:
            ax = sns.boxplot(x='MONTO', data=df_ayuntamiento)
            ax.xaxis.set_major_formatter(CustomFormatter())
            plt.title(f'Box Plot de Gastos para {ayuntamiento}', fontsize=16)
            plt.xlabel('Gastos', fontsize=14)
            plt.ylabel('')
            plt.xticks(rotation=45)
        else:
            plt.title(f'Box Plot de Gastos para {ayuntamiento} (Datos no disponibles)', fontsize=16)
            plt.xlabel('Gastos', fontsize=14)
            plt.xticks(rotation=45)
            plt.axis('off')
    plt.tight_layout()

    
    filename = f'graphs_ayuntamientos_boxplotParte3/{str(contador_imagenes).zfill(2)}.jpg'
    plt.savefig(filename)
    plt.close()


In [12]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd

ruta_gastos = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/Gastosos 2.xlsx'
gastos_df = pd.read_excel(ruta_gastos)
gastos_df = gastos_df.rename(columns={'GASTOS': 'MONTO'})


sns.set(style="whitegrid", palette="muted")


def money_formatter(x, pos):
    if x >= 1e6:  
        return '${:,.1f}M'.format(x * 1e-6)
    elif x >= 1e3:  
        return '${:,.0f}K'.format(x * 1e-3)
    else:
        return '${:,.0f}'.format(x)


ayuntamientos = gastos_df['AYUNTAMIENTO'].unique()


if not os.path.exists('graphs_ayuntamientos_CONCEPTO'):
    os.makedirs('graphs_ayuntamientos_CONCEPTO')


gastos_df['CONCEPTO'] = pd.Categorical(gastos_df['CONCEPTO'])

for i, ayuntamiento in enumerate(ayuntamientos, start=1):
    df_ayuntamiento = gastos_df[gastos_df['AYUNTAMIENTO'] == ayuntamiento]
    monthly_expenses = df_ayuntamiento.groupby('CONCEPTO')['MONTO'].sum().reset_index()

    plt.figure(figsize=(12, 6))
    bar_plot = sns.barplot(y='CONCEPTO', x='MONTO', data=monthly_expenses)  
    plt.title(f'Gastos por cocepto {ayuntamiento}', fontsize=16)
    plt.xlabel('Gasto', fontsize=14)  
    plt.ylabel('Concepto', fontsize=14)  
    plt.xticks(rotation=45)

    
    plt.gca().xaxis.set_major_formatter(mticker.FuncFormatter(money_formatter))  

    
    for p in bar_plot.patches:
        width = p.get_width()  
        bar_plot.annotate(money_formatter(width, None),
                          (width, p.get_y() + p.get_height() / 2.),  
                          ha='center', va='center',
                          xytext=(9, 0),  
                          textcoords='offset points')

    plt.tight_layout()

  
    filename = f'{str(i).zfill(2)}.jpg'  
    plt.savefig(f'graphs_ayuntamientos_CONCEPTO/{filename}')
    plt.close()


C:\Users\franc\AppData\Local\Temp\ipykernel_196\2326545641.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_expenses = df_ayuntamiento.groupby('CONCEPTO')['MONTO'].sum().reset_index()
C:\Users\franc\AppData\Local\Temp\ipykernel_196\2326545641.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_expenses = df_ayuntamiento.groupby('CONCEPTO')['MONTO'].sum().reset_index()
C:\Users\franc\AppData\Local\Temp\ipykernel_196\2326545641.py:40: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current be

In [5]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import numpy as np

ruta_gastos1 = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/Gastosos 2.xlsx'
gastos_df = pd.read_excel(ruta_gastos1)
gastos_df = gastos_df.rename(columns={'GASTOS': 'MONTO'})

sns.set(style="whitegrid", palette="muted")

class CustomFormatter(mticker.Formatter):
    def __call__(self, x, pos=None):
        if x >= 1e6:
            s = '{:1.1f}M'.format(x*1e-6)
        elif x >= 1e3:
            s = '{:1.0f}K'.format(x*1e-3)
        else:
            s = '{:1.0f}'.format(x)
        return s
    


def money_formatter(x, pos):
    if x >= 1e6:  
        return '${:,.1f}M'.format(x * 1e-6)
    elif x >= 1e3:  
        return '${:,.0f}K'.format(x * 1e-3)
    else:
        return '${:,.0f}'.format(x)

if not os.path.exists('graphs_ayuntamientos_boxplotParte4'):
    os.makedirs('graphs_ayuntamientos_boxplotParte')

ayuntamientos = gastos_df['AYUNTAMIENTO'].unique()
ayuntamientos_a_omitir = [
    'Junta de Distrito Municipal de Arroyo Dulce', 
    'Junta de Distrito Municipal de Platanal'
]

contador_imagenes = 0
for ayuntamiento in ayuntamientos:
    contador_imagenes += 1  
    
    plt.figure(figsize=(12, 6))
    if ayuntamiento in ayuntamientos_a_omitir:
        plt.title(f'Box Plot de Gastos para {ayuntamiento}', fontsize=16)
        plt.axis('off')  
    else:
        df_ayuntamiento = gastos_df[gastos_df['AYUNTAMIENTO'] == ayuntamiento]
        if not df_ayuntamiento.empty:
            ax = sns.boxplot(x=np.log(gastos_df['MONTO'] + 1), data=df_ayuntamiento)
            ax.xaxis.set_major_formatter(CustomFormatter())
            plt.title(f'Box Plot expansión de {ayuntamiento}', fontsize=16)
            plt.xlabel('Gastos', fontsize=14)
            plt.ylabel('')
            plt.xticks(rotation=45)
        else:
            plt.title(f'Box Plot de Gastos para {ayuntamiento} (Datos no disponibles)', fontsize=16)
            plt.xlabel('Gastos', fontsize=14)
            plt.xticks(rotation=45)
            plt.axis('off')
    plt.tight_layout()

    
    filename = f'graphs_ayuntamientos_boxplotParte4/{str(contador_imagenes).zfill(2)}.jpg'
    plt.savefig(filename)
    plt.close()


c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\franc\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc,

In [1]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import numpy as np

ruta_gastos1 = 'C:/Users/franc/OneDrive - INTEC/Escritorio/Proyectos/Python/Gobiernos locales/Gastosos 2.xlsx'
gastos_df = pd.read_excel(ruta_gastos1)
gastos_df = gastos_df.rename(columns={'GASTOS': 'MONTO'})

sns.set(style="whitegrid", palette="muted")


In [33]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import os


def millions_and_thousands(x, pos):
    if x >= 1e6:  
        return f'{x*1e-6:.1f}M'
    elif x >= 1e3:  
        return f'{x*1e-3:.0f}K'
    return f'{x:.0f}'


if not os.path.exists('graphs_ayuntamientos_Pareto'):
    os.makedirs('graphs_ayuntamientos_Pareto')


ayuntamientos = gastos_df['AYUNTAMIENTO'].unique()

contador_imagenes = 0

for ayuntamiento in ayuntamientos:
    contador_imagenes += 1  

    df_temp = gastos_df[gastos_df['AYUNTAMIENTO'] == ayuntamiento]
    pareto_df = df_temp.groupby('CONCEPTO')['MONTO'].sum().reset_index()
    pareto_df.sort_values('MONTO', ascending=False, inplace=True)
    pareto_df['Porcentaje'] = 100 * (pareto_df['MONTO'] / pareto_df['MONTO'].sum())
    pareto_df['Porcentaje_acumulado'] = pareto_df['Porcentaje'].cumsum()

    fig, ax = plt.subplots(figsize=(12, 6))

   
    for i, row in pareto_df.iterrows():
        color = 'C2' if row['Porcentaje_acumulado'] <= 80 else 'C0'
        ax.bar(row['CONCEPTO'], row['MONTO'], color=color)

    ax2 = ax.twinx()
    ax2.plot(pareto_df['CONCEPTO'], pareto_df['Porcentaje_acumulado'], color='C1', marker='o', linewidth=2)

   
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(millions_and_thousands))

    ax.set_title(f'Diagrama de Pareto de Gastos por Concepto para {ayuntamiento}')
    ax.set_xlabel('Concepto')
    ax.set_ylabel('Gasto')
    ax2.set_ylabel('Porcentaje acumulado (%)')
    ax.set_xticklabels(pareto_df['CONCEPTO'], rotation=45, ha='right')


    filename = f'graphs_ayuntamientos_Pareto/{str(contador_imagenes).zfill(2)}.jpg'
    plt.savefig(filename, bbox_inches='tight')  
    plt.close()

C:\Users\franc\AppData\Local\Temp\ipykernel_25720\4281592033.py:53: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(pareto_df['CONCEPTO'], rotation=45, ha='right')
C:\Users\franc\AppData\Local\Temp\ipykernel_25720\4281592033.py:53: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(pareto_df['CONCEPTO'], rotation=45, ha='right')
C:\Users\franc\AppData\Local\Temp\ipykernel_25720\4281592033.py:53: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(pareto_df['CONCEPTO'], rotation=45, ha='right')
C:\Users\franc\AppData\Local\Temp\ipykernel_25720\4281592033.py:53: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_